## Treinamento do Modelo

In [2]:
import tensorflow as tf

def resize_with_padding(img):
    img = tf.image.resize_with_pad(img, 224, 224)
    return img

In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# TensorFlow and keras
import tensorflow as tf
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from keras._tf_keras.keras.callbacks import EarlyStopping

dataset_path = '../dataset/training'

datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=resize_with_padding,
    validation_split=0.15,
    rotation_range=30,
    zoom_range=0.3,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss='binary_crossentropy', metrics=['accuracy'])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6
)
history = model.fit(train_data, validation_data=val_data, epochs=50, callbacks=[early_stop, reduce_lr])


model.save('model.keras')

## Resultados

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

# Accuracy

fig, ax = plt.subplots()
# Lines and colors 
ax.plot(history.history['accuracy'], label='Acurácia treino', color='#7281EE')
ax.plot(history.history['val_accuracy'], label='Acurácia validação', color='#D286EA')
# Title and labels
ax.set_title('Acurácia durante o treino', color='white')
ax.set_xlabel('Épocas', color='white')
ax.set_ylabel('Acurácia', color='white')
ax.tick_params(colors='white')
# Legend
ax.legend()
# Grid
ax.grid(True, color='white', linestyle='--', linewidth=0.2)
# Background
ax.set_facecolor('#242424')
fig.patch.set_facecolor('#242424')
# Border
for spine in ax.spines.values():
    spine.set_visible(False)
# 'y' format
ax.yaxis.set_major_formatter(PercentFormatter(1.0))

plt.show()

# Accuracy

fig, ax = plt.subplots()
# Lines and colors 
ax.plot(history.history['loss'], label='Perda treino', color='#7281EE')
ax.plot(history.history['val_loss'], label='Perda validação', color='#D286EA')
# Title and labels
ax.set_title('Perda durante o treino', color='white')
ax.set_xlabel('Épocas', color='white')
ax.set_ylabel('Loss', color='white')
ax.tick_params(colors='white')
# Legend
ax.legend()
# Grid
ax.grid(True, color='white', linestyle='--', linewidth=0.2)
# Background
ax.set_facecolor('#242424')
fig.patch.set_facecolor('#242424')
# Border
for spine in ax.spines.values():
    spine.set_visible(False)
# 'y' format
ax.yaxis.set_major_formatter(PercentFormatter(1.0))

plt.show()
